First two classes are copied from the provided lab script:

In [27]:
class dtree:
    """ A basic Decision Tree"""

    def __init__(self):
        """ Constructor """

    def read_data(self, filename):
        fid = open(filename, "r")
        data = []
        d = []
        for line in fid.readlines():
            d.append(line.strip())
        for d1 in d:
            data.append(d1.split(","))
        fid.close()

        self.featureNames = data[0]
        self.featureNames = self.featureNames[:-1]
        data = data[1:]
        self.classes = []
        for d in range(len(data)):
            self.classes.append(data[d][-1])
            data[d] = data[d][:-1]

        return data, self.classes, self.featureNames

    def classify(self, tree, datapoint):

        if type(tree) == type("string"):
            # Have reached a leaf
            return tree
        else:
            a = list(tree.keys())[0]
            for i in range(len(self.featureNames)):
                if self.featureNames[i] == a:
                    break

            try:
                t = tree[a][datapoint[i]]
                return self.classify(t, datapoint)
            except:
                return None

    def classifyAll(self, tree, data):
        results = []
        for i in range(len(data)):
            results.append(self.classify(tree, data[i]))
        return results

    def make_tree(self, data, classes, featureNames, maxlevel=-1, level=0, forest=0):
        """ The main function, which recursively constructs the tree"""

        nData = len(data)
        nFeatures = len(data[0])

        try:
            self.featureNames
        except:
            self.featureNames = featureNames

        # List the possible classes
        newClasses = []
        for aclass in classes:
            if newClasses.count(aclass) == 0:
                newClasses.append(aclass)

        # Compute the default class (and total entropy)
        frequency = np.zeros(len(newClasses))

        totalEntropy = 0
        totalGini = 0
        index = 0
        for aclass in newClasses:
            frequency[index] = classes.count(aclass)
            totalEntropy += self.calc_entropy(float(frequency[index]) / nData)
            totalGini += (float(frequency[index]) / nData) ** 2

            index += 1

        totalGini = 1 - totalGini
        default = classes[np.argmax(frequency)]

        if nData == 0 or nFeatures == 0 or (maxlevel >= 0 and level > maxlevel):
            # Have reached an empty branch
            return default
        elif classes.count(classes[0]) == nData:
            # Only 1 class remains
            return classes[0]
        else:

            # Choose which feature is best
            gain = np.zeros(nFeatures)
            ggain = np.zeros(nFeatures)
            featureSet = list(range(nFeatures))
            if forest != 0:
                np.random.shuffle(featureSet)
                featureSet = featureSet[0:forest]
            for feature in featureSet:
                g, gg = self.calc_info_gain(data, classes, feature)
                gain[feature] = totalEntropy - g
                ggain[feature] = totalGini - gg

            bestFeature = np.argmax(gain)
            tree = {featureNames[bestFeature]: {}}

            # List the values that bestFeature can take
            values = []
            for datapoint in data:
                if datapoint[feature] not in values:
                    values.append(datapoint[bestFeature])

            for value in values:
                # Find the datapoints with each feature value
                newData = []
                newClasses = []
                index = 0
                for datapoint in data:
                    if datapoint[bestFeature] == value:
                        if bestFeature == 0:
                            newdatapoint = datapoint[1:]
                            newNames = featureNames[1:]
                        elif bestFeature == nFeatures:
                            newdatapoint = datapoint[:-1]
                            newNames = featureNames[:-1]
                        else:
                            newdatapoint = datapoint[:bestFeature]
                            newdatapoint.extend(datapoint[bestFeature + 1:])
                            newNames = featureNames[:bestFeature]
                            newNames.extend(featureNames[bestFeature + 1:])
                        newData.append(newdatapoint)
                        newClasses.append(classes[index])
                    index += 1

                # Now recurse to the next level
                subtree = self.make_tree(newData, newClasses, newNames, maxlevel, level + 1, forest)

                # And on returning, add the subtree on to the tree
                tree[featureNames[bestFeature]][value] = subtree

            return tree

    def printTree(self, tree, name):
        if type(tree) == dict:
            print(name, list(tree.keys())[0])
            for item in list(list(tree.values())[0].keys()):
                print(name, item)
                self.printTree(list(tree.values())[0][item], name + "\t")
        else:
            print
            name, "\t->\t", tree

    def calc_entropy(self, p):
        if p != 0:
            return -p * np.log2(p)
        else:
            return 0

    def calc_info_gain(self, data, classes, feature):

        # Calculates the information gain based on both entropy and the Gini impurity
        gain = 0
        ggain = 0
        nData = len(data)

        # List the values that feature can take

        values = []
        for datapoint in data:
            if datapoint[feature] not in values:
                values.append(datapoint[feature])

        featureCounts = np.zeros(len(values))
        entropy = np.zeros(len(values))
        gini = np.zeros(len(values))
        valueIndex = 0
        # Find where those values appear in data[feature] and the corresponding class
        for value in values:
            dataIndex = 0
            newClasses = []
            for datapoint in data:
                if datapoint[feature] == value:
                    featureCounts[valueIndex] += 1
                    newClasses.append(classes[dataIndex])
                dataIndex += 1

            # Get the values in newClasses
            classValues = []
            for aclass in newClasses:
                if classValues.count(aclass) == 0:
                    classValues.append(aclass)

            classCounts = np.zeros(len(classValues))
            classIndex = 0
            for classValue in classValues:
                for aclass in newClasses:
                    if aclass == classValue:
                        classCounts[classIndex] += 1
                classIndex += 1

            for classIndex in range(len(classValues)):
                entropy[valueIndex] += self.calc_entropy(float(classCounts[classIndex]) / np.sum(classCounts))
                gini[valueIndex] += (float(classCounts[classIndex]) / np.sum(classCounts)) ** 2

            # Computes both the Gini gain and the entropy
            gain = gain + float(featureCounts[valueIndex]) / nData * entropy[valueIndex]
            ggain = ggain + float(featureCounts[valueIndex]) / nData * gini[valueIndex]
            valueIndex += 1
        return gain, 1 - ggain

In [28]:
class bagger:
    """The bagging algorithm based on the decision tree of Chapter 6"""

    def __init__(self):
        """ Constructor """
        self.tree = dtree()

    def bag(self, data, targets, features, nSamples):

        nPoints = np.shape(data)[0]
        nDim = np.shape(data)[1]
        self.nSamples = nSamples

        # Compute bootstrap samples
        samplePoints = np.random.randint(0, nPoints, (nPoints, nSamples))
        classifiers = []

        for i in range(nSamples):
            sample = []
            sampleTarget = []
            for j in range(nPoints):
                sample.append(data[samplePoints[j, i]])
                sampleTarget.append(targets[samplePoints[j, i]])
            # Train classifiers
            classifiers.append(self.tree.make_tree(sample, sampleTarget, features, 1))

        return classifiers

    def bagclass(self, classifiers, data):

        decision = []
        # Majority voting
        for j in range(len(data)):
            outputs = []
            # print data[j]
            for i in range(self.nSamples):
                out = self.tree.classify(classifiers[i], data[j])
                if out is not None:
                    outputs.append(out)
            # List the possible outputs
            out = []
            for each in outputs:
                if out.count(each) == 0:
                    out.append(each)
            frequency = np.zeros(len(out))

            index = 0
            if len(out) > 0:
                for each in out:
                    frequency[index] = outputs.count(each)
                    index += 1
                decision.append(out[frequency.argmax()])
            else:
                decision.append(None)
        return decision

Class Random_Forest main difference in comparison with Bagging is a a splitting method. In Bagging we select all features without any thought, and in Random Forest we select a set of features which are used to create the tree which provides the best classification based on the given feature, after testing and selecting the best one.
This selection can be based on gini index, entropy or both.

Another important difference, is that the size of a classifier (size of forest) is based on a number of trees, not a number of samples.

In [42]:
from random import randrange

class Random_Forest:
    """The Random Forest algorithm based on the decision tree of Chapter 6"""

    def __init__(self):
        """ Constructor """
        self.tree = dtree()
    
    """Create samples with replacement from datatset"""
    def subsample(self, dataset, classes, ratio):
        sample = list()
        self.sampleClass = list()
        self.nSamples = round(len(dataset) * ratio)
        while len(sample) < self.nSamples:
            index = randrange(len(dataset))
            self.sampleClass.append(classes[index])
            sample.append(dataset[index])
        return sample

    def build_forest(self, data, classes, features, sample_size, forest_size):
        
        sample = self.subsample( data, classes, sample_size )
        
        self.Forest_size = forest_size
        
        classifiers = []
        
        for i in range(self.Forest_size):
            # Train classifiers
            classifiers.append(self.tree.make_tree(sample, self.sampleClass, features, 1, 0, 4))

        return classifiers

    

    def forest_prediction(self, classifiers, data):

        decision = []
        # Majority voting
        for j in range(len(data)):
            outputs = []
            # print data[j]
            for i in range(self.Forest_size):
                out = self.tree.classify(classifiers[i], data[j])
                if out is not None:
                    outputs.append(out)
            # List the possible outputs
            out = []
            for each in outputs:
                if out.count(each) == 0:
                    out.append(each)
            frequency = np.zeros(len(out))

            index = 0
            if len(out) > 0:
                for each in out:
                    frequency[index] = outputs.count(each)
                    index += 1
                decision.append(out[frequency.argmax()])
            else:
                decision.append(None)
        return decision

In [ ]:
import numpy as np

tree = dtree()
bag  = bagger()

data,classes,features = tree.read_data('car.data')

train = data[::2][:]
test = data[1::2][:]
trainc = classes[::2]
testc = classes[1::2]

rdforest = Random_Forest()
classifiers=rdforest.build_forest(train,trainc,features, 1.0, 50)
out = rdforest.forest_prediction(classifiers,test)
 
a = np.zeros(len(out))
b = np.zeros(len(out))
d = np.zeros(len(out))
 
for i in range(len(out)):
    if testc[i] == 'good' or testc[i]== 'v-good':
        b[i] = 1
        if out[i] == testc[i]:
            d[i] = 1
    if out[i] == testc[i]:
        a[i] = 1
print("-----")
print("Forest")
print("Number correctly predicted",str(np.sum(a)))
print("Number of testpoints ",str(len(a)))
print("Percentage Accuracy ",str(np.sum(a)/len(a)*100.0))

The increase of accuracy comes with the cost of performance and size of the forest. Bigger forest can yield better results, but takes significantly more time to calculate.  

For 10 trees, the accuracy is circa 74%, and for 20 trees the accuracy is circa 78%. The latter result is comparable to what we got in the Bagging method. 

Based on: https://machinelearningmastery.com/implement-random-forest-scratch-python/
Done by Marcin Wadowski, parasited by Artur Kepka.